In [1]:
import pandas as pd
import numpy as np
import dash
import flask
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from pandas_datareader import data as web
from datetime import datetime as dt

In [2]:
import pymongo

In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
password = os.getenv("PASSWORD")
username = os.getenv("USERNAME")

In [4]:
client = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client["1-grams"]

In [5]:
"""
import csv
wordlist = []
with open('wordlist.csv', 'rt') as f:
    reader = csv.reader(f)
    wordlist = list(reader)
"""

"\nimport csv\nwordlist = []\nwith open('wordlist.csv', 'rt') as f:\n    reader = csv.reader(f)\n    wordlist = list(reader)\n"

In [ ]:
def load(word):
    try:
        df = pd.DataFrame(list(db["tweets"].find({ "word": word})))
        df = df.sort_values(by=['time'])
        df['year'] = [date.year for date in df['time']]
        df['day'] = [date.timetuple().tm_yday for date in df['time']]
        return df
    except:
        print(str("No timeseries found for "+word))
        df = pd.DataFrame()
        return df

In [ ]:
load('@johnh')

,_id,counts,rank,time,word,year,day
2,5c44bb447ad7ce0434482749,35,316116,2012-07-02,@johnh,2012,184
0,5c4478c47ad7ce04345ce376,30,601464,2015-03-07,@johnh,2015,66
1,5c4478ec7ad7ce04346dce9c,40,491848,2015-03-09,@johnh,2015,68
3,5c4957c17ad7ce3a999fbd41,11,1236819,2017-05-28,@johnh,2017,148


# Input box for query

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    dcc.Input(id='my-id', value='@realdonaldtrump', type='text'),
    dcc.Graph(id='my-graph'),
])

@app.callback(Output('my-graph', 'figure'), [Input('my-id', 'value')])
def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    values = input_value.split(',')
    for item in values:
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server(port=8050, host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)
10.245.186.62 - - [18/Apr/2019 12:19:32] "GET / HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 12:19:32] "GET /styles.css HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 12:19:33] "GET /_dash-layout HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 12:19:33] "GET /_dash-dependencies HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 12:19:34] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 12:19:58] "GET / HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 12:19:58] "GET /styles.css HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 12:19:59] "GET /_dash-layout HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 12:19:59] "GET /_dash-dependencies HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 12:19:59] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.182.216 - - [18/Apr/2019 12:21:12] "GET / HTTP/1.1" 200 -
10.245.182.216 - - [18/Apr/2019 12:21:12] "GET /styles.css HTTP/1.1" 200 -
10.245.182.216 - - [18/Apr/2019 12:2

No timeseries found for 


10.245.182.216 - - [18/Apr/2019 12:21:30] "POST /_dash-update-component HTTP/1.1" 200 -
[2019-04-18 12:21:29,419] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2656, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1601, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1608, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'time'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/flask/app.py", line 2292, in wsgi_app


No timeseries found for taco 
No timeseries found for taco s


...

10.245.182.216 - - [18/Apr/2019 12:24:06] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.182.216 - - [18/Apr/2019 12:24:07] "POST /_dash-update-component HTTP/1.1" 500 -
10.245.182.216 - - [18/Apr/2019 12:24:07] "POST /_dash-update-component HTTP/1.1" 500 -
10.245.182.216 - - [18/Apr/2019 12:24:07] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.182.216 - - [18/Apr/2019 12:24:07] "POST /_dash-update-component HTTP/1.1" 200 -
[2019-04-18 12:24:07,647] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2656, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1601, in pandas._libs.hashtable.PyObjectHashTable.get_

No timeseries found for @bts+t


[2019-04-18 12:24:08,046] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2656, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1601, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1608, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'time'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/janeadams/.conda/envs/test_env

No timeseries found for @bts+


10.245.182.216 - - [18/Apr/2019 12:24:09] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.182.216 - - [18/Apr/2019 12:24:09] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.182.216 - - [18/Apr/2019 12:24:09] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.182.216 - - [18/Apr/2019 12:24:09] "POST /_dash-update-component HTTP/1.1" 200 -
10.245.182.216 - - [18/Apr/2019 12:24:10] "POST /_dash-update-component HTTP/1.1" 200 -
132.198.138.54 - - [18/Apr/2019 12:27:22] "GET /_favicon.ico HTTP/1.1" 200 -
132.198.138.54 - - [18/Apr/2019 12:27:22] "GET /_dash-layout HTTP/1.1" 200 -
132.198.138.54 - - [18/Apr/2019 12:27:23] "POST /_dash-update-component HTTP/1.1" 200 -


# Dropdown for query

In [ ]:
import numpy as np

In [ ]:
from numpy import random

In [ ]:
nums = random.randint(0,len(wordlist),size=10000)

In [ ]:
options = []
for num in nums:
    word = wordlist[num][0]
    options.append({'label': word,'value': word})

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.Div([
        dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','covfefe'], multi=True)]),
    dcc.Graph(id='my-graph')
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])

def update_graph(input_value):
    data=[]
    figure = {'data':data}
    for item in input_value:
        df = load(item)
        data.append({'x':df['time'], 'y':df['counts'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server()